# Prepare data

In [ ]:
from causalml.dataset import make_uplift_classification

# Define the treatment names based on the 6 treatments mentioned in the paper
treatment_names = ['control', 'treatment_1', 'treatment_2', 'treatment_3', 'treatment_4', 'treatment_5', 'treatment_6']

delta_uplift_increase = {
    'control': 0.0, # <--- Uplift của Control = 0
    'treatment_1': 0.05, 'treatment_2': 0.1, 'treatment_3': 0.12,
    'treatment_4': 0.17, 'treatment_5': 0.2, 'treatment_6': 0.2
}

delta_uplift_decrease = {
    'control': 0.0, # <--- Uplift của Control = 0
    'treatment_1': 0.01, 'treatment_2': 0.02, 'treatment_3': 0.03,
    'treatment_4': 0.05, 'treatment_5': 0.06, 'treatment_6': 0.07
}
n_uplift_increase_mix_informative = [0, 1, 2, 3, 4, 5, 6]
n_uplift_decrease_mix_informative = [0, 1, 1, 1, 1, 1, 1]
    
# Generate the data
df, x_names = make_uplift_classification(
    n_samples=10000 * 7, # Adjust total samples to account for control + 6 treatments
    treatment_name=treatment_names,
    n_classification_features=100,
    n_classification_informative=20,
    n_classification_redundant=10,
    n_classification_repeated=10,
    positive_class_proportion=0.2,
    delta_uplift_increase_dict=delta_uplift_increase,
    delta_uplift_decrease_dict=delta_uplift_decrease,
    n_uplift_increase_mix_informative_dict={t: v for t, v in zip(treatment_names, n_uplift_increase_mix_informative)},
    n_uplift_decrease_mix_informative_dict={t: v for t, v in zip(treatment_names, n_uplift_decrease_mix_informative)},
    random_seed=42
)

X = df[x_names].values

Y = df['conversion'].values

treatment_map = {
    'control': 0,
    'treatment_1': 1,
    'treatment_2': 2,
    'treatment_3': 3,
    'treatment_4': 4,
    'treatment_5': 5,
    'treatment_6': 6
}
T = df['treatment_group_key'].map(treatment_map).values

# Verify shapes
print(f"X shape: {X.shape}")
print(f"Y shape: {Y.shape}")
print(f"T shape: {T.shape}")

Failed to import duecredit due to No module named 'duecredit'


X shape: (420000, 100)
Y shape: (420000,)
T shape: (420000,)


In [5]:
print(df['treatment_group_key'].value_counts())

treatment_group_key
treatment_2    70000
treatment_6    70000
treatment_4    70000
treatment_3    70000
treatment_5    70000
treatment_1    70000
Name: count, dtype: int64


# Objective function for *optuna*

In [4]:
from model.end_to_end_pipeline import Dragonnet
from sklearn.model_selection import train_test_split
import optuna
import torch
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklift.metrics import uplift_auc_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def objective_inner_cv(trial, X, y, t):
    """
    Hàm mục tiêu cho Optuna. 
    Thực hiện Inner Cross-Validation để đánh giá bộ tham số.
    """
    # Hidden size & Outcome size là multiplier trên input_dim [cite: 843-844]
    hidden_ratio = trial.suggest_float("hidden_ratio", 0.5, 2.0)
    outcome_ratio = trial.suggest_float("outcome_ratio", 0.5, 2.0)
    
    alpha = trial.suggest_float("alpha", 0.5, 1.5)
    beta = trial.suggest_float("beta", 0.5, 1.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256]) # Tăng batch size cho ổn định
    
    # Tính toán kích thước layer
    input_dim = X.shape[1]
    shared_hidden = int(input_dim * hidden_ratio)
    outcome_hidden = int(input_dim * outcome_ratio)

    # === B. Inner Cross-Validation Loop (5 folds theo paper) ===
    # Để tiết kiệm thời gian demo, tôi để n_splits=3. Bạn hãy đổi thành 5 để giống hệt paper.
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    
    auuc_scores = []
    
    # Stratify theo Treatment để đảm bảo mỗi fold đều có đủ Control/Treatment
    for train_idx, val_idx in skf.split(X, t):
        X_train_inner, X_val_inner = X[train_idx], X[val_idx]
        y_train_inner, y_val_inner = y[train_idx], y[val_idx]
        t_train_inner, t_val_inner = t[train_idx], t[val_idx]
        
        # Khởi tạo model
        model = Dragonnet(
            input_dim=input_dim,
            shared_hidden=shared_hidden,
            outcome_hidden=outcome_hidden,
            alpha=alpha,
            beta=beta,
            epochs=30, # Paper limit [cite: 837]
            batch_size=batch_size,
            learning_rate=learning_rate,
            loss_type='tarreg',
            device=device,
            seed=42
        )
        
        # Train (có thể dùng 1 phần validation nhỏ để early stopping hoặc train full epoch)
        # Ở inner loop này ta train full 30 epochs cho đơn giản và nhanh
        model.fit(X_train_inner, y_train_inner, t_train_inner, valid_perc=None)
        
        # Predict trên tập Validation của Inner fold
        y0_pred, y1_pred, t_pred, eps = model.predict(X_val_inner)
        uplift_pred = (y1_pred - y0_pred).flatten()
        
        # Tính metric: Uplift AUC
        score = uplift_auc_score(
            y_true=y_val_inner.flatten(),
            uplift=uplift_pred,
            treatment=t_val_inner.flatten()
        )
        auuc_scores.append(score)
        
    # Trả về trung bình AUUC của các fold
    return np.mean(auuc_scores)

In [5]:
target_treatments = [1, 2, 3, 4, 5, 6] 

for treat_id in target_treatments:
    print(f"\n{'='*20} PROCESSING TREATMENT {treat_id} {'='*20}")
    
    # --- A. Lọc dữ liệu ---
    mask = np.isin(T, [0, treat_id])
    X_sub = X[mask]
    Y_sub = Y[mask]
    T_sub = T[mask]
    
    # Map Treatment ID về 1
    T_sub_binary = np.where(T_sub == treat_id, 1, 0)
    
    # --- B. Outer Cross-Validation ---
    outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Danh sách để lưu kết quả và params
    outer_results = [] 
    best_params_history = [] # <--- NEW: List lưu best params của từng fold
    
    fold_idx = 0
    for train_idx, test_idx in outer_cv.split(X_sub, T_sub_binary):
        fold_idx += 1
        print(f"\n--- Outer Fold {fold_idx}/5 ---")
        
        X_outer_train, X_outer_test = X_sub[train_idx], X_sub[test_idx]
        y_outer_train, y_outer_test = Y_sub[train_idx], Y_sub[test_idx]
        t_outer_train, t_outer_test = T_sub_binary[train_idx], T_sub_binary[test_idx]
        
        # --- C. Hyperparameter Tuning (TPE) ---
        print("  -> Running TPE Search (Inner CV)...")
        study = optuna.create_study(direction="maximize")
        study.optimize(
            lambda trial: objective_inner_cv(trial, X_outer_train, y_outer_train, t_outer_train), 
            n_trials=20, 
            show_progress_bar=False # Tắt progress bar con để đỡ rối
        )
        
        best_params = study.best_params
        print(f"  -> Best Params (Fold {fold_idx}): {best_params}")
        
        # --- D. Refit Final Model ---
        print("  -> Refitting Final Model...")
        input_dim = X_outer_train.shape[1]
        final_model = Dragonnet(
            input_dim=input_dim,
            shared_hidden=int(input_dim * best_params['hidden_ratio']),
            outcome_hidden=int(input_dim * best_params['outcome_ratio']),
            alpha=best_params['alpha'],
            beta=best_params['beta'],
            epochs=30,
            batch_size=best_params['batch_size'],
            learning_rate=best_params['learning_rate'],
            loss_type='tarreg',
            device=device,
            seed=42
        )
        
        final_model.fit(X_outer_train, y_outer_train, t_outer_train, valid_perc=None)
        
        # --- E. Evaluate ---
        y0_test_pred, y1_test_pred, _, _ = final_model.predict(X_outer_test)
        uplift_test_pred = (y1_test_pred - y0_test_pred).flatten()
        
        final_auuc = uplift_auc_score(
            y_true=y_outer_test.flatten(),
            uplift=uplift_test_pred,
            treatment=t_outer_test.flatten()
        )
        
        outer_results.append(final_auuc)
        print(f"  -> Fold {fold_idx} Test AUUC: {final_auuc:.4f}")
        
        # --- LƯU LẠI PARAMS VÀ KẾT QUẢ ---
        best_params_history.append({
            'fold': fold_idx,
            'auuc': final_auuc,
            'params': best_params
        })

    # --- KẾT QUẢ TỔNG HỢP CHO TREATMENT ---
    mean_auuc = np.mean(outer_results)
    std_auuc = np.std(outer_results)
    
    print(f"\n{'#'*60}")
    print(f"SUMMARY FOR TREATMENT {treat_id}")
    print(f"Mean AUUC = {mean_auuc:.4f} ± {std_auuc:.4f}")
    
    # Tìm bộ tham số có kết quả Test cao nhất (Best of the Best)
    best_run = max(best_params_history, key=lambda x: x['auuc'])
    
    print("\n--- Best Parameters found in each Fold ---")
    for item in best_params_history:
        print(f"Fold {item['fold']} (AUUC {item['auuc']:.4f}): {item['params']}")
        
    print("\n--- SUGGESTED BEST PARAMS (From highest scoring fold) ---")
    print(f"From Fold {best_run['fold']} with AUUC {best_run['auuc']:.4f}:")
    print(best_run['params'])
    print(f"{'#'*60}\n")

[I 2025-12-20 12:14:14,391] A new study created in memory with name: no-name-b18b2f1f-6482-475b-b1d7-d74b6de1e1b8



==================== PROCESSING TREATMENT 1 ====================

--- Outer Fold 1/5 ---
  -> Running TPE Search (Inner CV)...


[W 2025-12-20 12:14:23,499] Trial 0 failed with parameters: {'hidden_ratio': 0.5848954436477455, 'outcome_ratio': 1.8267322330109301, 'alpha': 1.4950664096373054, 'beta': 0.8664309261505964, 'learning_rate': 0.0007825794776104228, 'batch_size': 64} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\ducha\miniconda3\Lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ducha\AppData\Local\Temp\ipykernel_6708\1801700088.py", line 35, in <lambda>
    lambda trial: objective_inner_cv(trial, X_outer_train, y_outer_train, t_outer_train),
                  ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ducha\AppData\Local\Temp\ipykernel_6708\2687006379.py", line 59, in objective_inner_cv
    model.fit(X_train_inner, y_train_inner, t_train_inner, valid_perc=None)
    ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 